**Введение**  
Здесь показан алгоритм обработки данных о предыдущих кандидатах компаннии, обучения модели на них и получения списка новых кандидатов, которых стоит рассмотреть для приглашения на собеседование  

**Важное примечание**  
Данные синтетические и используются лишь для примера  
В анкетах работников есть и текстовые данные о навыках и опыте, которые путём заполнения электронной анкеты просто преобразовать в табличные  
Работу с текстом CatBoost берёт на себя

Установим библиотеки, которые обычно не установлены по умолчанию

In [1]:
!pip install -U catboost
!pip install -U feature_selector --no-dependencies

Requirement already up-to-date: catboost in /usr/local/lib/python3.6/dist-packages (0.24.4)
Requirement already up-to-date: feature_selector in /usr/local/lib/python3.6/dist-packages (1.0.0)


Import'им библиотеки и методы

In [2]:
%%time
%pylab inline
import pandas as pd
import numpy as np
import catboost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from feature_selector import FeatureSelector

Populating the interactive namespace from numpy and matplotlib
CPU times: user 324 ms, sys: 141 ms, total: 464 ms
Wall time: 356 ms


In [3]:
#checking versions
print(catboost.__version__)
!python --version

0.24.4
Python 3.6.9


Загрузим данные о прошлых кандидатах некого работодателя и посмотрим, что они из себя представляют

In [4]:
addr_hr_1_train = "./data/hr_1_train.csv"
addr_hr_1_test = "./data/hr_1_test.csv"

In [5]:
data_train = pd.read_csv(addr_hr_1_train).drop(columns = ["hired", "id"])
target = pd.read_csv(addr_hr_1_train)["hired"]
data_test = pd.read_csv(addr_hr_1_test)

In [6]:
data_train.shape

(2000, 5)

In [7]:
data_test.shape

(899, 6)

In [8]:
data_train.head()

,gender,age,has_car,children,salary
0,1.0,32,0.0,1,180000
1,1.0,20,1.0,0,250000
2,0.0,24,NaN,0,310000
3,0.0,42,1.0,1,110000
4,0.0,25,1.0,1,300000


In [9]:
data_train.describe()

,gender,age,has_car,children,salary
count,1932.000000,2000.000000,1860.000000,2000.00000,2.000000e+03
mean,0.507764,98.411500,0.501075,0.32300,9.039300e+05
std,0.500069,361.675137,0.500133,0.55931,4.396999e+06
min,0.000000,19.000000,0.000000,0.00000,0.000000e+00
25%,0.000000,23.000000,0.000000,0.00000,1.000000e+05
50%,1.000000,26.000000,1.000000,0.00000,2.000000e+05
75%,1.000000,31.000000,1.000000,1.00000,3.000000e+05
max,1.000000,2001.000000,1.000000,3.00000,3.900000e+07


Избавимся от ошибок в анкете (кто-то вместо возраста указал год рождения)

In [10]:
data_train["age"] = data_train["age"].apply(lambda x: x if x < 200 else 2021-x).astype(int)
data_test["age"] = data_train["age"].apply(lambda x: x if x < 200 else 2021-x).astype(int)

Увидим положительные изменения в данных

In [11]:
data_train.describe()[["age"]]

,age
count,2000.000000
mean,29.441500
std,13.161998
min,19.000000
25%,23.000000
50%,26.000000
75%,31.000000
max,121.000000


Избавимся от пропущенных значений, подставляя вместо них медиану по ряду

In [12]:
to_fill = ["gender", "age", "has_car", "children", "salary"]
for col in to_fill:
  median_train = data_train[col].median()
  data_train[col] = data_train[col].fillna(median_train)
  median_test = data_test[col].median()
  data_test[col] = data_test[col].fillna(median_test)

Вновь увидим положительные изменения в данных

In [13]:
data_train.describe()

,gender,age,has_car,children,salary
count,2000.000000,2000.000000,2000.000000,2000.00000,2.000000e+03
mean,0.524500,29.441500,0.536000,0.32300,9.039300e+05
std,0.499524,13.161998,0.498827,0.55931,4.396999e+06
min,0.000000,19.000000,0.000000,0.00000,0.000000e+00
25%,0.000000,23.000000,0.000000,0.00000,1.000000e+05
50%,1.000000,26.000000,1.000000,0.00000,2.000000e+05
75%,1.000000,31.000000,1.000000,1.00000,3.000000e+05
max,1.000000,121.000000,1.000000,3.00000,3.900000e+07


Нормализуем числовые данные линейным методом, т.е. распределим их на отрезке [0;1] по формуле x = (x - min)/(max - min)

In [14]:
to_normalize = ["age", "salary", "children"]
for col in to_normalize:
  mx = max(data_train[col].max(), data_test[col].max())
  mn = min(data_train[col].min(), data_test[col].min())
  dif = mx - mn
  data_train[col] = data_train[col].apply(lambda x: (x - mn) / dif)
  data_test[col] = data_test[col].apply(lambda x: (x - mn) / dif)

In [15]:
data_train.describe()

,gender,age,has_car,children,salary
count,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.524500,0.102368,0.536000,0.080750,0.023178
std,0.499524,0.129039,0.498827,0.139827,0.112744
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.039216,0.000000,0.000000,0.002564
50%,1.000000,0.068627,1.000000,0.000000,0.005128
75%,1.000000,0.117647,1.000000,0.250000,0.007692
max,1.000000,1.000000,1.000000,0.750000,1.000000


Выделим категориальные фичи для дальнейшей передачи в CatBoost и преобразуем их в целочисленный тип

In [16]:
cat_features = ["has_car", "gender"]
for col in cat_features:
  data_train[col] = data_train[col].astype(int)
  data_test[col] = data_test[col].astype(int)

Так как используем CatBoost, можем не беспокоиться о текстовых фичах, их векторизуют за нас

Разделим обучающую выборку на основную и валидационную в соотношении 70%/30%. Параметром stratify разделим данные так, чтобы соотношение классов в основной и валидационной выборках было равным

In [17]:
x_train, x_validation, y_train, y_validation = train_test_split(data_train,
                                                                target,
                                                                test_size=0.3,
                                                                stratify = target)

Создаем классификатор, устанавливаем гиперпараметры, подбирая их в зависимости от размера датасета и колонок  
При необходимости можно воспользоваться методами GridSearch и RandomSearch по гиперпараметрам  
Методы также доступны в библиотеке CatBoost  
Не забываем передать категориальные фичи в качестве аргумента

In [18]:
model = CatBoostClassifier(iterations=500,
                            depth = 2,
                            learning_rate = 0.005,
                            # l2_leaf_reg = 4,
                            eval_metric="F1",
                            loss_function = "Logloss",
                            task_type="GPU",
                            # fold_permutation_block = 2,
                            # fold_len_multiplier = 1.5,
                            # leaf_estimation_iterations = 10,
                            # max_ctr_complexity = 1,
                            random_seed= 127,
                            cat_features = cat_features
                           )

Обучаем модель, используя валидационную выборку, устанавливаем ограничение early_stopping_rounds для избежания переобучения

In [19]:
model.fit(x_train, 
          y_train, 
          eval_set=(x_validation, y_validation), 
          use_best_model=True, 
          early_stopping_rounds=300,  
          plot=False, 
          verbose=100
          )

0:	learn: 0.6908213	test: 0.7155425	best: 0.7155425 (0)	total: 18ms	remaining: 9.01s
100:	learn: 0.9150327	test: 0.9083969	best: 0.9083969 (32)	total: 1.49s	remaining: 5.89s
200:	learn: 0.9150327	test: 0.9083969	best: 0.9083969 (32)	total: 2.96s	remaining: 4.4s
300:	learn: 0.9240711	test: 0.9207547	best: 0.9207547 (235)	total: 4.53s	remaining: 2.99s
400:	learn: 0.9240711	test: 0.9207547	best: 0.9207547 (235)	total: 6.03s	remaining: 1.49s
499:	learn: 0.9240711	test: 0.9207547	best: 0.9207547 (235)	total: 7.53s	remaining: 0us
bestTest = 0.920754717
bestIteration = 235
Shrink model to first 236 iterations.


Получаем predict probability

In [20]:
pred = model.predict_proba(data_test.drop(columns = ["id"]))

In [21]:
test_ids = data_test["id"].tolist()
best_candidates = []

Выбираем для просмотра только тех кандидатов, у которых принадлежность ко второму классу больше порогового значения

In [22]:
for i in range(len(test_ids)):
  if (pred[i][1] > 0.65):
    best_candidates.append(test_ids[i])

Получаем id кандидатов

In [23]:
len(best_candidates)

229

In [24]:
best_candidates

[3025,
 3034,
 3037,
 3038,
 3045,
 3060,
 3062,
 3064,
 3068,
 3076,
 3081,
 3087,
 3089,
 3093,
 3102,
 3104,
 3126,
 3144,
 3148,
 3154,
 3161,
 3162,
 3177,
 3185,
 3189,
 3203,
 3245,
 3248,
 3259,
 3267,
 3274,
 3292,
 3296,
 3301,
 3303,
 3304,
 3319,
 3322,
 3330,
 3340,
 3348,
 3353,
 3370,
 3374,
 3393,
 3394,
 3398,
 3401,
 3411,
 3427,
 3445,
 3448,
 3449,
 3453,
 3455,
 3474,
 3484,
 3485,
 3489,
 3493,
 3497,
 3501,
 3503,
 3521,
 3526,
 3549,
 3552,
 3557,
 3566,
 3569,
 3571,
 3597,
 3613,
 3616,
 3623,
 3628,
 3645,
 3647,
 3653,
 3674,
 3694,
 3702,
 3703,
 3711,
 3712,
 3713,
 3723,
 3736,
 3772,
 3777,
 3779,
 3789,
 3791,
 3796,
 3797,
 3801,
 3807,
 3827,
 3853,
 3860,
 3865,
 3866,
 3881,
 3883,
 3897,
 3909,
 3910,
 3927,
 3931,
 3932,
 3938,
 3951,
 3967,
 3969,
 3998,
 3999,
 4021,
 4023,
 4025,
 4026,
 4035,
 4039,
 4052,
 4056,
 4061,
 4064,
 4069,
 4071,
 4094,
 4121,
 4124,
 4126,
 4142,
 4149,
 4153,
 4167,
 4168,
 4171,
 4172,
 4175,
 4198,
 4208,
 4214,

При необходимости мы можем посмотреть "важность" фичей и убрать из датасетов те фичи, которые не влияют на ответ модели. Иногда это приносит пользу.

In [25]:
model.get_feature_importance(prettified=True, verbose = True).head(20)

,Feature Id,Importances
0,has_car,49.571656
1,children,22.849013
2,salary,20.251699
3,age,7.327632
4,gender,0.000000


In [ ]:
useless_features = model.get_feature_importance(prettified=True, verbose = True).query("Importances==0")["Feature Id"].tolist()
len(useless_features)

In [ ]:
data_train = data_train.drop(columns = useless_features)

In [ ]:
data_test = data_test.drop(columns = useless_features)